### Installation

In [ ]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unslot"یادآور",
            "تماس کارشناس",
            "تقسیط",
            "پیشنهاد پوشش افزوده",
            "بن باشگاه",h") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 8192  # gpt-oss-20B fully supports 8k-12k context with Unsloth, 2048k is safe and useful for your long JSONs
dtype = None           # Let Unsloth auto-detect (will pick bfloat16 on A100/H100, float16 on RTX 4090 etc.)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",                  # This is the correct & latest Unsloth version (MXFP4)
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,                                 # Uses Q4_K_M by default → best quality/speed trade-off
    # token = None,                                      # add your HF token only if you hit rate limits
)

# This line is VERY important for full fine-tuning (not just LoRA) on Unsloth
model = FastLanguageModel.get_peft_model(
    model,
    r=64,                    # 32 or 64 is good for 20B, 64 gives slightly better quality
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64,
    lora_dropout = 0,        # currently only supports 0
    bias = "none",
    use_gradient_checkpointing = "unsloth",  # saves HUGE VRAM, highly recommended
    random_state = 3407,
    use_rslora = True,       # [2025 best practice] Rank-Stabilized LoRA, better than standard LoRA
    loftq_config = None,     # leave None
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.37G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth: Making `model.base_model.model.model` require gradients


### Reasoning Effort
The `gpt-oss` models from OpenAI include a feature that allows users to adjust the model's "reasoning effort." This gives you control over the trade-off between the model's performance and its response speed (latency) which by the amount of token the model will use to think.

----

The `gpt-oss` models offer three distinct levels of reasoning effort you can choose from:

* **Low**: Optimized for tasks that need very fast responses and don't require complex, multi-step reasoning.
* **Medium**: A balance between performance and speed.
* **High**: Provides the strongest reasoning performance for tasks that require it, though this results in higher latency.

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "medium", # **NEW!** Set reasoning effort to low, medium or high
).to("cuda")

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-11-29

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>The prompt: "Solve x^5 + 3x^4 - 10 = 3." Probably means solve the equation x^5 + 3x^4 - 10 = 3 for x? But maybe they meant solve for x in the equation x^5 + 3x


Changing the `reasoning_effort` to `medium` will make the model think longer. We have to increase the `max_new_tokens` to occupy the amount of the generated tokens but it will give better and more correct answer

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "medium", # **NEW!** Set reasoning effort to low, medium or high
).to("cuda")

_ = model.generate(**inputs, max_new_tokens = 128, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-11-29

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>The user says: "Solve x^5 + 3x^4 - 10 = 3." So we have to solve equation: x^5 + 3x^4 - 10 = 3. Wait, check: the equation is something like x^5 + 3x^4 - 10 = 3. So we need to find x that satisfy this. Let's rewrite: x^5 + 3x^4 - 10 = 3. Bring 3 to left: x^5 + 3x^4 - 10 - 3 = 0 => x


Lastly we will test it using `reasoning_effort` to `high`

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high", # **NEW!** Set reasoning effort to low, medium or high
).to("cuda")

_ = model.generate(**inputs, max_new_tokens = 128, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-11-29

Reasoning: high

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>We have an equation: "Solve x^5 + 3x^4 - 10 = 3." Wait, that's strange: the equation is x^5 + 3x^4 - 10 = 3. We need to solve for x. So rewrite: x^5 + 3x^4 - 10 = 3. Bring all terms to left: x^5 + 3x^4 - 10 - 3 = 0 => x^5 + 3x^4 - 13 = 0. We need to solve for x, presumably find real roots


<a name="Data"></a>
### Data Prep

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

# 1. Set the correct ChatML template (gpt-oss-20B was trained on ChatML)
"""tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",      # perfect match
    mapping = {"role": "role", "content": "content"},   # default, no need to rename user/assistant
)"""

# 2. Define the Formatting Function
def formatting_prompts_func(examples):
    convos = examples["messages"]
    # tokenize=False: We want the string format first
    # add_generation_prompt=False: We are training, so we include the assistant's answer in the text
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts }

# 3. Load Your Dataset
SFT_PATH = "path"

dataset = load_dataset(
    "path",
    data_files = SFT_PATH,
    split = "train",
)

# 4. Apply formatting and drop all original columns (keeps only "text")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    num_proc=8,
    remove_columns=dataset.column_names,   # important: only keep the formatted "text" column
)

# 5. Optional but recommended: train/validation split
dataset = dataset.train_test_split(test_size=0.02, seed=3407)  # 2 % is enough for 20B

# 6. Final sanity check
print(f"Total examples   : {len(dataset['train'])}")
print(f"Validation examples: {len(dataset['test'])}")
print("\nFirst formatted example:\n")
print(dataset["train"][0]["text"])

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=8):   0%|          | 0/1609 [00:00<?, ? examples/s]

Total examples   : 1576
Validation examples: 33

First formatted example:

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-11-29

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

شما دستیار ارشد صدور و نگهداشت بیمه آتش‌سوزی هستید. با توجه به risk_score و ویژگی‌های مشتری و احتمال ریزش، تحلیل ریسک و برنامه نگهداشت مرحله‌بندی‌شده ارائه کنید و فقط JSON استاندارد برگردانید.<|end|><|start|>user<|message|>{"customer_profile": {"years_with_company": null, "total_premium_paid": null, "renewal_count": null, "avg_claim_amount": null, "claim_count": null, "days_since_first_policy": null, "days_since_last_renewal": null, "days_to_expiration": null}, "underwriting": {"risk_score": 74, "risk_level": "بالا", "coverage": null}, "churn": {"p_chu

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    max_seq_length=8192,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,          # effective batch size ≈ 32
        warmup_steps=10,
        num_train_epochs=2,                   # change to 1 or 3 as you wish
        learning_rate=5e-5,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="path",
        report_to="none",
        neftune_noise_alpha=5,                      # great for Persian + JSON
    ),
    # ← FIXED: renamed 'evaluation_strategy' → 'eval_strategy' (2025 API)
    training_args=TrainingArguments(
        per_device_eval_batch_size=2,
        eval_strategy="steps",                     # ← this was the culprit
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        prediction_loss_only=True,
    ),
)

# Quick check
print("Sample input →\n", trainer.train_dataset[0]["text"][:1000])

# GO!
trainer_stats = trainer.train()

# Save final LoRA
model.save_pretrained("path")
tokenizer.save_pretrained("path")

print("Training finished! LoRA adapters saved.")

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1576 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/33 [00:00<?, ? examples/s]

Sample input →
 <|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-11-29

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

شما دستیار ارشد صدور و نگهداشت بیمه آتش‌سوزی هستید. با توجه به risk_score و ویژگی‌های مشتری و احتمال ریزش، تحلیل ریسک و برنامه نگهداشت مرحله‌بندی‌شده ارائه کنید و فقط JSON استاندارد برگردانید.<|end|><|start|>user<|message|>{"customer_profile": {"years_with_company": null, "total_premium_paid": null, "renewal_count": null, "avg_claim_amount": null, "claim_count": null, "days_since_first_policy": null, "days_since_last_renewal": null, "days_to_expiration": null}, "underwriting": {"risk_score": 74, "risk_level": "بالا", "coverage": null}, "churn": {"p_churn": 0.0, "segment": "Low", "top_reasons": []}, "constraint

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,576 | Num Epochs = 2 | Total steps = 198
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 31,850,496 of 20,946,607,680 (0.15% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,10.548800
2,10.187700
3,7.574000
4,4.797900
5,3.786600
6,3.072700
7,2.723600
8,2.423100
9,2.236700
10,2.054400


Training finished! LoRA adapters saved.


In [10]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-11-29\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nشما دستیار ارشد صدور و نگهداشت بیمه آتش\u200cسوزی هستید. با توجه به risk_score و ویژگی\u200cهای مشتری و احتمال ریزش، تحلیل ریسک و برنامه نگهداشت مرحله\u200cبندی\u200cشده ارائه کنید و فقط JSON استاندارد برگردانید.<|end|><|start|>user<|message|>{"customer_profile": {"years_with_company": null, "total_premium_paid": null, "renewal_count": null, "avg_claim_amount": null, "claim_count": null, "days_since_first_policy": null, "days_since_last_renewal": null, "days_to_expiration": null}, "underwriting": {"risk_score": 80, "risk_level": "بالا", "coverage": "هزینه پاکسازی"}, "churn": {"p_churn": 0.0, "segment": "Low", "top

<a name="Inference"></a>
### Inference

In [ ]:
import json
import torch
from transformers import TextStreamer

# No need to call get_chat_template again — the tokenizer already knows ChatML perfectly
# (calling it again with wrong mapping can break things)

# Your test case
input_block = {
    "customer_profile": {
      ""
    },
    "underwriting": {
      ""
    },
    "churn": {
      ""
    },
    "constraints": {
      ""
    }
}

messages = [
    {"role": "system", "content": ""},
    {"role": "user",   "content": json.dumps(input_block, ensure_ascii=False, indent=None)}
]

# Apply ChatML template correctly
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,      
    return_tensors="pt"
).to("cuda")

# برای دیدن خروجی لحظه‌ای
text_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=False)

print("=== خروجی مدل ===")
_ = model.generate(
    input_ids=inputs,
    max_new_tokens=2048,
    temperature=0.0,          
    top_p=1.0,
    do_sample=False,
    repetition_penalty=1.2,
    streamer=text_streamer,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

=== خروجی مدل ===
<|message|>{"summary": "ریسک متوسط برای ساختمان مسکونی با تراکم خطر پایین اما ترکیدگی لوله. پذیرش مشروط با افزایش جزئی حق بیمه.", "retention_plan": [{"step": 1, "action": "بازدید کارشناسی قبل از صدور"}], "underwriting_notes": ["با وجود طبقه خطر ۱ و نوع سازه بتنی که مقاومت بالایی دارد، تراکم خطر پایین منطقه و عدم تراکم خطر در همان ساختمان باعث شده تا ریسک کلی متوسط شود."], "cx_message_short": "ریسک متوسط برای ساختمان مسکونی با تراکم خطر پایین اما ترکیدگی لوله. پذیرش مشروط با افزایش جزئی حق بیمه."}<|return|>


In [ ]:
# Optional: push to Hugging Face Hub
model.push_to_hub("mahdishahsavari/gpt-oss-20B-finetune", token="")
tokenizer.push_to_hub("mahdishahsavari/gpt-oss-20B-finetune", token="")

README.md:   0%|          | 0.00/604 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          |  560kB /  127MB            

Saved model to https://huggingface.co/mahdishahsavari/gpt-oss-20B-finetune


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpadcw7ydz/tokenizer.json: 100%|##########| 27.9MB / 27.9MB            

In [ ]:
"""if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "finetuned_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 1024,
        dtype = None,
        load_in_4bit = True,
    )

messages = [
    {"role": "system", "content": "reasoning language: French\n\nYou are a helpful assistant that can solve mathematical problems."},
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high",
).to("cuda")
from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))"""

### Saving to float16 for VLLM or mxfp4

We also support saving to `float16` or `mxfp4` directly. Select `merged_16bit` for float16. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# ------------------------------------------------------------------------------
# 6) Optional: merge LoRA and export (merged / GGUF)
# ------------------------------------------------------------------------------

if True:  # Change to True to save merged finetuned model (single full model)
    model.save_pretrained_merged("gpt-oss-20B-finetune-merged", tokenizer)

if True:  # Change to True to upload merged finetuned model
    model.push_to_hub_merged(
        "mahdishahsavari/gpt-oss-20B-finetune-merged",
        tokenizer,
        token = "",
    )

if True:  # Change to True to save GGUF for Ollama / llama.cpp
    model.save_pretrained_gguf(
        "gpt-oss-20B-finetune-gguf",
        tokenizer,
        quantization_method = "Q8_0",  # Supports Q8_0, BF16, F16
    )

if True:  # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "mahdishahsavari/gpt-oss-20B-finetune-gguf",
        tokenizer,
        quantization_method = "Q8_0",
        token = "",
    )

# Merge and push to hub in mxfp4
if True:
    model.save_pretrained_merged("gpt-oss-20B-finetune-merged-mxfp4", tokenizer, save_method = "mxfp4")
if True: model.push_to_hub_merged("mahdishahsavari/gpt-oss-20B-finetune-merged-mxfp4", tokenizer, token = "", save_method = "mxfp4")

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00000-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 3/3 [01:20<00:00, 26.72s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 3/3 [04:22<00:00, 87.48s/it]


Unsloth: Regenerating safetensors index for dequantized MXFP4 model...
Unsloth: Merge process complete. Saved to `/content/gpt-oss-20B-finetune-merged`
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00000-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

Unsloth: Merge process complete. Saved to `/content/gpt-oss-20B-finetune-merged-mxfp4`
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00000-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 3/3 [01:30<00:00, 30.02s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into mxfp4: 100%|██████████| 3/3 [02:47<00:00, 55.69s/it]


Unsloth: Merge process complete. Saved to `/content/mahdishahsavari/gpt-oss-20B-finetune-merged-mxfp4`
